
- overall goal:
    - specific ROI targetting w/ minimal campaigns
- what this means for accnt structure
    - for many variables we must split campaigns to target ROI
    - want to capture greatest amt of rps variation w/ 
        fewest # of campaigns
- 2 ways of approaching this
    1. minimize rps variation w/in a campaign
        - i.e. after campaign split want to minimize rps variance w/in campaigns
        => i actually think this is eq to decision tree regression w/ split criterion MSE
        https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html
        - tried this out a little - but the computation was intensive which made it slow going
    2. maximize rps variation outside campaigns
    - i.e. after campaign split want to minimize rps variance w/in campaigns
        => i actually think this is eq to decision tree regression w/ split criterion MSE
        https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html
    - mostly focused on this approach

- approaches I tried
1. rank splits by their correlation/covaraince/MI w/ `agg_rps`
    - tried ANOVA,pt.biserial,MI - had most success w/ MI
    - was dificult to compute b/c couldnt find methods that accepted sample weight
    - MI approach was promising but didnt go down that route
2. rank splits by intra-split agg_rps variance
    - had most success w/ this
    - tested out an approahc where I cluster the split on rps - had good results
    - think this is the mtd to use going forward
    TODO:
    - test fitting on general traffic sources?
    - test clustering on multiple days of rps data - 
        or rolling rps data or something
    - try out more granular aggs
    - is there some kind of metric which measures total variance
        and computes how much of that variance is captured by a split?

3. fit decision tree on rps data 
    - MSE criterion is apparently the same as minimizng inter split variance
    - computationally intensive
    - not 100% clear how to go from tree to campaign structure


- can do this by
    1. choosing what vars or tuples of vars to split campaigns on
    2. grouping similar buckets w/in those splits
- want to measure correlation/dependence of categorical split vars w/ cont rps
    - cat,cat metrics w/ binned rps
        chi
    - cont,cont metrcs w/ 1 hot encoded split vars
    - cat,cont metrics
- correlation metrics
    - MI:
        - sklearn.feature_selection.mi_regression
        - would need AFAICT to rresample input arrays
    - ANOVA:
        - scipy.stats.f_oneway
    - pt biserial
        - needs binary vars tho
        - https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pointbiserialr.html
        - https://towardsdatascience.com/point-biserial-correlation-with-python-f7cd591bd3b1
- want most variation of rps w/ fewest camapaigns
- dont want to group similar buckets together - think its too complicated


TODO: 2021-05-20
  - test fitting on general traffic sources?
  - test clustering on multiple days of rps data - 
      or rolling rps data or something
  - try out more granular aggs
  - is there some kind of metric which measures total variance
      and computes how much of that variance is captured by a split?

Trevor: 2021-05-21
TODO:
- figure out how to minimize campaign # when writing back to taboola
    - want to make sure campaigns have sufficient traffic
- kw=(location,os,device)
- calc 30-day rps/kw
- can create distribution over the rps(kw) distribution
- 100 campaigns - 1 per percentile

TODO: 2021-05-24
- look into clustering each individual split variable

In [1]:
import sys
import re
import os
# detect if we are running from a `notebooks/*_shared` folder
# re.match("not.*shared",sys.path[0])
if sys.path[0].endswith("_shared"):
    sys.path[0] = "/".join(sys.path[0].split("/")[:-2])
assert sys.path[0].endswith("adtech")

from utils.env import load_env_from_aws
load_env_from_aws()

import pprint
from IPython.display import display as ipydisp    
import collections
import itertools
import datetime

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from models.utils import wavg,get_wavg_by
from notebooks.aduriseti_shared.utils import *

NOW = datetime.datetime.now()
DAY = datetime.timedelta(days=1)

campaign_split_fields = dict(
    # traffic_source = ["traffic_source"],
    browser = ["browser"],
    operating_system = ["operating_system"],
    device = ["device"],
    # channel = ["channel"],
    # domain = ["domain"],
    product = ["product"],
    # keyword = ["keyword"],
    # campaign_id = ["campaign_id"],
    # landing_page = ["landing_page"],
    TOD = ["dayofweek","hourofday"],
    dma = ["dma"],
    state =["state",],
    location = ["state","dma"],
    
    dma_os=["dma", "operating_system"],
    dma_device=["dma", "device", ],
    dma_os_device=["dma", "operating_system", "device"],

    state_os=["state", "operating_system"],
    state_device=["state", "device", ],
    state_os_device=["state", "operating_system", "device"],

    location_os = ["state", "dma", "operating_system"],
    location_device=["state", "dma", "device", ],
    location_os_device = ["state", "dma", "operating_system","device"],
)

def get_wthresh(W,p):
    W = rps_df["sessions"].sort_values(ascending=False)
    Wsum = W.sum()
    cumsum = 0
    for wthresh in W:
        if cumsum > Wsum * p:
            break
        cumsum += wthresh
    return wthresh

start_date = NOW - 90*DAY
eval_date = NOW - 30*DAY
end_date = NOW

split2aggrps = {}
for split,split_cols in campaign_split_fields.items():
    print(split,split_cols)
    rps_df = agg_rps(start_date,end_date,None,traffic_source=TABOOLA,agg_columns=tuple(split_cols+["utc_dt"]))
    rps_df = translate_taboola_vals(rps_df)
    rps_df["split_on"] = split
    split2aggrps[split] = rps_df
    print(split,rps_df.shape)

Attempting to load environ from: `SM_ENV_BASE`
...Success!!
browser ['browser']


INFO:root:query: SELECT browser,utc_dt, COUNT(session_id)                    ... executed in 4.47 seconds
INFO:numexpr.utils:Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


browser (94, 11)
operating_system ['operating_system']


INFO:root:query: SELECT operating_system,utc_dt, COUNT(session_id)           ... executed in 4.43 seconds


operating_system (340, 11)
device ['device']


...

location_os (17620, 11)
location_device ['state', 'dma', 'device']


INFO:root:query: SELECT state,dma,device,utc_dt, COUNT(session_id)           ... executed in 4.91 seconds


location_device (15343, 11)
location_os_device ['state', 'dma', 'operating_system', 'device']


INFO:root:query: SELECT state,dma,operating_system,device,utc_dt, COUNT(sessi... executed in 5.03 seconds


location_os_device (17881, 11)


In [42]:
import sklearn.preprocessing
import sklearn.cluster
import sklearn.tree

class AggRPSClust:
    def __init__(self,minclust=8):
        self.minclust = minclust
        self.split_size = None
    
    def fit(self,X,_):
        globals()["X"] = X
        assert X.index.names[-1] == "utc_dt"
        split_idx = X.reset_index()[X.index.names[:-1]] .apply(tuple,axis=1).apply(str)
        self.split_size = split_idx.drop_duplicates().__len__()
        if self.split_size <= self.minclust:
            self.ord_enc = sklearn.preprocessing.OrdinalEncoder(
                    handle_unknown="use_encoded_value",unknown_value=-1) \
                .fit(split_idx.drop_duplicates().values.reshape(-1,1))
            self.ord_enc.transform(split_idx.values.reshape(-1,1))
        else:
            self.aggX = X \
                .groupby(X.index.names[:-1]) \
                .agg({
                    "sessions": sum,
                    "rps": get_wavg_by(X,"sessions")
                })

            # nclust = MINCLUST
            nclust = max(self.minclust, int(np.log(self.split_size)))
            # nclust = max(MINCLUST,int(split_size ** 0.5))
            # print("nclust", nclust, split_size, np.log(split_size))
            
            self.aggX['clust'] = sklearn.cluster.KMeans(n_clusters=nclust) \
                            .fit_predict(self.aggX[["rps"]],sample_weight=self.aggX["sessions"])

        return self

    def transform(self,X):
        globals()["X"] = X
        assert X.index.names[-1] == "utc_dt"
        split_idx = X.reset_index()[X.index.names[:-1]] .apply(tuple,axis=1).apply(str)
        if self.split_size <= self.minclust:
            return self.ord_enc.transform(split_idx.values.reshape(-1,1))
        else:
            X["clust"] = 1
            X["clust"] *= self.aggX["clust"]
            return X["clust"]
        
    def fit_transform(self,X,_):
        raise NotImplementedError
        

        
# class AggRPSClustBad:
#     def __init__(self,minclust=8):
#         self.minclust = minclust
#         self.split_size = None
    
#     def fit(self,X,_):
#         globals()["X"] = X
#         assert X.index.names[-1] == "utc_dt"
#         split_idx = X.reset_index()[X.index.names[:-1]] .apply(tuple,axis=1).apply(str)
#         self.split_size = split_idx.drop_duplicates().__len__()
#         if self.split_size <= self.minclust:
#             self.ord_enc = sklearn.preprocessing.OrdinalEncoder(
#                     handle_unknown="use_encoded_value",unknown_value=-1) \
#                 .fit(split_idx.drop_duplicates().values.reshape(-1,1))
#             self.ord_enc.transform(split_idx.values.reshape(-1,1))
#         else:
#             # nclust = MINCLUST
#             nclust = max(self.minclust, int(np.log(self.split_size)))
#             # nclust = max(MINCLUST,int(split_size ** 0.5))
#             # print("nclust", nclust, split_size, np.log(split_size))
            
#             aggX = X \
#                 .groupby(X.index.names[:-1]) \
#                 .agg({
#                     "sessions": sum,
#                     "rps": get_wavg_by(X,"sessions")
#                 })
#             self.clusterer = sklearn.cluster.KMeans(n_clusters=self.minclust) \
#                 .fit(aggX[["rps"]],sample_weight=aggX["sessions"])

#         return self

#     def transform(self,X):
#         globals()["X"] = X
#         assert X.index.names[-1] == "utc_dt"
#         split_idx = X.reset_index()[X.index.names[:-1]] .apply(tuple,axis=1).apply(str)
#         if self.split_size <= self.minclust:
#             return self.ord_enc.transform(split_idx.values.reshape(-1,1))
#         else:
#             aggX = X \
#                 .groupby(X.index.names[:-1]) \
#                 .agg({
#                     "sessions": sum,
#                     "rps": get_wavg_by(X,"sessions")
#                 })
#             aggX["clust"] = self.clusterer \
#                 .predict(aggX[["rps"]],sample_weight=aggX["sessions"])
#             X["clust"] = 1
#             X["clust"] *= aggX["clust"]
#             return X["clust"]
        
#     def fit_transform(self,X,_):
#         raise NotImplementedError

from models.utils import wstd
def get_split_factor(rps_df):
    split_attr2unique_vals = {c: rps_df.index.unique(c) for c in rps_df.index.names[:-1]}
    _,new_index_order = zip(*sorted((V.__len__(),c) for c,V in split_attr2unique_vals.items()))
    return rps_df.reset_index()[[*new_index_order[:-1],"clust"]].drop_duplicates().__len__()

In [43]:
perfD = []
for split,rps_df in split2aggrps.items():
    # rps_df["lps"] = rps_df["num_leads"] / rps_df["sessions"]
    # rps_df["rpl"] = rps_df["revenue"] / rps_df["num_leads"]
    # rps_df["score"] = rps_df[["score_null_avg","score_adv_avg","score_supp_avg"]].sum(axis=1)
    rps_df["rps"] = rps_df["rps_avg"]
    fitI = rps_df.reset_index()['utc_dt'] < eval_date
    fitI.index = rps_df.index

    clusterer = AggRPSClust().fit(rps_df[fitI],None)
    rps_df["clust"] = -1
    rps_df.loc[fitI,"clust"] = clusterer.transform(rps_df[fitI])
    rps_df.loc[~fitI,"clust"] = clusterer.transform(rps_df[~fitI])

    agg_rps_df = rps_df \
        [~fitI] \
        .groupby(rps_df.index.names[:-1]) \
        .agg({
            "sessions": sum,
            "rps": get_wavg_by(rps_df[~fitI],"sessions")
        })
    clust_rps_df = rps_df \
        [~fitI] \
        .groupby("clust") \
        .agg({
            "sessions": sum,
            "rps": get_wavg_by(rps_df[~fitI],"sessions")
        })

    assert clust_rps_df["rps"].max() <= agg_rps_df["rps"].max()
    rps_wavg = wavg(agg_rps_df[["rps"]], agg_rps_df["sessions"])
    rps_clust_wavg = wavg(clust_rps_df[["rps"]], clust_rps_df["sessions"])
    assert all((rps_wavg - rps_clust_wavg).abs() < 1e-2), (rps_wavg, rps_clust_wavg)


    perfd = {
        "split": split,
        "fit_shape": agg_rps_df.shape,
        "clust_shape": clust_rps_df.shape,
        "split_variance": wstd(agg_rps_df["rps"], agg_rps_df["sessions"]),
        "cluster_variance": wstd(clust_rps_df["rps"], clust_rps_df["sessions"]),
        # wstd(rps_df["rps_avg"],rps_df["sessions"])
        "clustered_split_factor": get_split_factor(rps_df),
    }
    perfD.append(perfd)
    pprint.pprint(perfd)
    ipydisp(clust_rps_df)

perfdf = pd.DataFrame(perfD)
ipydisp(perfdf)

{'clust_shape': (2, 2),
 'cluster_variance': 0.008213534619983488,
 'clustered_split_factor': 2,
 'fit_shape': (2, 2),
 'split': 'browser',
 'split_variance': 0.008213534619983488}


,sessions,rps
clust,,
0.0,7,0.00000
1.0,25000,0.49099


{'clust_shape': (6, 2),
 'cluster_variance': 0.08133980743537546,
 'clustered_split_factor': 6,
 'fit_shape': (6, 2),
 'split': 'operating_system',
 'split_variance': 0.08133980743537546}


,sessions,rps
clust,,
0.0,12,0.000000
1.0,882,0.227959
2.0,1194,0.392755
3.0,22476,0.514690
4.0,312,0.023558
5.0,130,0.224769


{'clust_shape': (3, 2),
 'cluster_variance': 0.08778434823763963,
 'clustered_split_factor': 3,
 'fit_shape': (3, 2),
 'split': 'device',
 'split_variance': 0.08778434823763963}


...

,sessions,rps
clust,,
0,5146,0.052300
1,4,25.237500
2,564,1.749894
4,7607,0.541412
5,148,3.521077
6,6541,0.287898
7,4997,0.879517


<ipython-input-42-b672701563b4>:99: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["clust"] = 1
<ipython-input-42-b672701563b4>:100: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["clust"] *= aggX["clust"]
<ipython-input-42-b672701563b4>:99: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a

{'clust_shape': (8, 2),
 'cluster_variance': 0.69054293625801,
 'clustered_split_factor': 623,
 'fit_shape': (1116, 2),
 'split': 'location_os_device',
 'split_variance': 0.7142755578236577}


,sessions,rps
clust,,
0,9271,0.108577
1,8,25.808750
2,5882,0.838799
3,7,12.761429
4,158,3.326832
5,500,1.752500
6,9118,0.470581
7,62,5.578548


,split,fit_shape,clust_shape,split_variance,cluster_variance,clustered_split_factor
0,browser,"(2, 2)","(2, 2)",0.008214,0.008214,2
1,operating_system,"(6, 2)","(6, 2)",0.081340,0.081340,6
2,device,"(3, 2)","(3, 2)",0.087784,0.087784,3
3,product,"(2, 2)","(2, 2)",0.003104,0.003104,2
4,TOD,"(522, 2)","(7, 2)",0.371523,0.359315,46
5,dma,"(209, 2)","(8, 2)",0.340472,0.331351,8
6,state,"(51, 2)","(8, 2)",0.205778,0.202136,8
7,location,"(339, 2)","(8, 2)",0.471682,0.433036,212
8,dma_os,"(689, 2)","(8, 2)",0.553719,0.535418,29
9,dma_device,"(451, 2)","(7, 2)",0.424180,0.410634,18


In [ ]:
# BEST_SPLIT = "location_os_device"
BEST_SPLIT = "dma_os_device"
# BEST_SPLIT = "dma_os"

rps_df_fit, rps_df_eval, rps_clust_df = cluster_split(BEST_SPLIT)
rps_df = rps_df_fit
split_attr2unique_vals = {index_col: rps_df.index.unique(index_col) 
                          for index_col in rps_df.index.names}
_,new_index_order = zip(*sorted((V.__len__(),c) for c,V in split_attr2unique_vals.items()))
campaign_df = rps_df \
    .groupby([*new_index_order[:-1], "clust"]) \
    .agg({
        "sessions": sum,
        "rps_avg": get_wavg_by(rps_df,"sessions"),
        new_index_order[-1]: tuple
    })
assert campaign_df["sessions"].sum() == rps_df_fit["sessions"].sum()
camp_rps_wavg = wavg(campaign_df["rps_avg"],campaign_df["sessions"])
fit_rps_wavg = wavg(rps_df_fit["rps_avg"], rps_df_fit["sessions"])
assert abs(camp_rps_wavg - fit_rps_wavg) < 1e-5

excl_campaign_df = campaign_df.groupby([*new_index_order[:-1]]) \
    .agg({
        new_index_order[-1]: tuple
    })
def flatten(M):
    return tuple(el for r in M for el in r)
excl_campaign_df[new_index_order[-1]] = excl_campaign_df[new_index_order[-1]] .apply(flatten)

camps = []
for idx,r in campaign_df.iterrows():
    camp = {
        "sessions_60d": r["sessions"], 
        "rps_avg_60d": r["rps_avg"]
    }
    for field,val in zip(new_index_order[:-1],idx):
        camp[field] = {"includes": val}
    last_field = new_index_order[-1]
    camp[last_field] = {
        "includes": r[last_field]
    }
    camps.append(camp)

for idx, r in excl_campaign_df.iterrows():
    camp = {}
    for field, val in zip(new_index_order[:-1], idx):
        camp[field] = {"includes": val}
    last_field = new_index_order[-1]
    camp[last_field] = {
        "excludes": r[last_field]
    }
    camps.append(camp)

camp_df = pd.DataFrame(camps)
camp_df.to_csv("campaign_dump.csv")

In [ ]:
import numpy as np
import pandas as pd
import scipy.stats
import sklearn.feature_selection
import sklearn.metrics
from matplotlib import pyplot as plt
import sklearn.preprocessing

requires_campaign_split = [
    # "browser",
    "operating_system",
    "device",
    # "channel", # not actually sure what this is
    # "domain",
    "product",
    # "landing_page",
    "location",
]

for split in requires_campaign_split:
    rps_df = split2aggrps[split] .reset_index()
    Xs = rps_df \
        .apply(lambda r: [r["int_ix"]]*int(r["sessions"]), axis=1)
    X = np.concatenate(Xs.values).reshape(-1,1)
    ys = rps_df \
        .apply(lambda r: [r["rps_avg"]]*int(r["sessions"]),axis=1)
    y = np.concatenate(ys.values)
    # print(y.min(),np.quantile(y, 0.5),y.max())
    y = y > y.mean()
    # y = np.concatenate(ys.values).reshape(-1,1)
    # y = sklearn.preprocessing.KBinsDiscretizer(n_bins=2,encode="ordinal") \
    #     .fit_transform(y).reshape(-1)
    mi = sklearn.feature_selection.mutual_info_regression(X,y,discrete_features=True)
    print(split,mi,y.mean(),rps_df.shape)

In [ ]:
import numpy as np
import pandas as pd
import scipy.stats
from matplotlib import pyplot as plt

requires_campaign_split = [
    "browser",
    "operating_system",
    "device",
    # "channel", # not actually sure what this is
    # "domain",
    "product",
    # "landing_page",
    "location",
]
combined_rps_df = pd.concat([df.reset_index() for df in split2aggrps.values()])
splitI = combined_rps_df["split_on"].isin(requires_campaign_split)
density = scipy.stats.gaussian_kde(
    dataset=combined_rps_df.loc[splitI,"rps_avg"],
    weights=combined_rps_df.loc[splitI, "sessions"],
    # bw_method="scott",
    # bw_method="silverman",
    # bw_method=0.1,
)
xs = np.linspace(0,2,100)
plt.plot(xs,density(xs))

In [ ]:
for split in requires_campaign_split:
    splitI = combined_rps_df["split_on"] == split
    density = scipy.stats.gaussian_kde(
        dataset=combined_rps_df.loc[splitI, "rps_avg"],
        weights=combined_rps_df.loc[splitI, "sessions"])
    xs = np.linspace(0, 2, 100)
    plt.plot(xs, density(xs))
    plt.title(split)
    plt.show()